# ESCAPA CON LA LLAVE - MANUAL
---
---

## Descripción de las clases
---

La representación del estado y de los niveles visto en el notebook de funciones básicas se ha organizado en clases.

La clase **State**: 
- tiene un constructor que recibe el jugador, la(s) piedra(s) y la(s) casilla(s) de agua. 
- tiene un método **get_rocks** que devuelve las piedras
- tiene un método **get_water** que devuelve las coordenadas con agua

- tiene un método **\_\_str\_\_** que permite obtener la representación en modo texto del objeto y un método **\_\_repr\_\_** equivalente, para usarlo dentro de colecciones.

- Tiene también un método que permite determinar si dos estados son iguales y también otro para calcular su *hash*.

La clase **Level**:
- tiene un constructor que recibe el tablero y los destinos
- tiene un método **get_board** que devuelve el tablero, y las paredes.
- tiene un método **get_target** que devuelve el destino al que se debe dirigir el jugador
- tiene un método **\_\_str\_\_** que permite obtener la representación en modo texto del objeto, así como un método **\_\_repr\_\_** equivalente

**Loader**, **UI**, **Mediator**

## Uso de tipados personalizados
---
* ```StaticObject = Tuple[int, int]```
* ```StateObject = Set[StaticObject]```
* ```StatePlayer = List[int]```
* ```LevelBoard = Tuple[Tuple[int]]```


In [1]:
# Cargar Celda

# Tipado en Python.
from JuegoEscapa import StateObject, LevelBoard, StaticObject, StatePlayer, Level, State

## Ejemplos
---

In [2]:
# Ejemplo de carga de nivel.

from Loader import Loader

loader = Loader()
files = loader.get_all_levels()
txt_level = loader.get_txt_level(files[3])

level, state = loader.load_level(txt_level)

In [3]:
# Obtener el tablero del nivel.

level.get_board()

((1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1),
 (1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1),
 (0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

In [4]:
# Ejemplo de visualización de un nivel.

from ipywidgets import HTML
from Gui import Gui

global state
ui = Gui()

html_str = ui.get_html(level, state)
HTML(value = html_str)

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

## Funciones
---
- move(level, state, mov): Recibe un nivel, un estado y un movimiento. **Devuelve un nuevo estado** resultante de aplicar el movimiento.

- is_goal(state): Recibe un estado. Devuelve True si el jugador es None.

**NOTA - IMPORTANTE**: No debes borrar **# Cargar Celda** en cada una de las celdas en que aparezca.

### > Funciones de validación

In [5]:
# Cargar Celda

def is_valid(board: LevelBoard,
             coord: StaticObject) -> bool:
    """
    Devuelve si las coordenadas son válidas en el tablero pasado.

    Args:
        board (LevelBoard): Tablero en el que comprobar la coordenada.
        coord (StaticObject): Coordenada a comprobar validez.

    Returns:
        bool: Devuelve si la coordenada está en el tablero.
    """
    try:
        x, y = coord
        return board[x][y] == 1
    except IndexError:
        return False


def is_rocks_free(board: LevelBoard,
                  rocks: StateObject,
                  coord: StaticObject):
    """
    Comprueba si la posición tiene rocas.

    Args:
        board (LevelBoard): Tablero en el que buscar.
        rocks (StateObject): Rocas.
        coord (StaticObject): Coordenadas en las que comprobar si la piedra puede moverse.

    Returns:
        bool: Devuelve si la posición tiene rocas.
    """
    return (is_valid(board, coord)
            and not tuple(coord) in rocks)


def is_cross_free(board: LevelBoard,
                  crosses: StateObject,
                  coord: StaticObject):
    """
    Comprueba si la posición tiene cruces.

    Args:
        board (LevelBoard): Tablero en el que buscar.
        crosses (StateObject): Cruces.
        coord (StaticObject): Coordenadas en las que comprobar si la piedra puede moverse.

    Returns:
        bool: Devuelve si la posición tiene cruces.
    """
    return (is_valid(board, coord) and not tuple(coord) in crosses)


def is_player_free(board: LevelBoard,
                   rocks: StateObject,
                   water: StateObject,
                   crosses: StateObject,
                   coord: StaticObject):
    """ Comprueba si la coordenada pasada puede ser pisada por el jugador.

    Args:
        board (LevelBoard): Tablero en el que buscar.
        rocks (StateObject): Rocas.
        water (StateObject): Agua.
        crosses (StateObject): Cruces.
        coord (StaticObject): Coordenadas en las que comprobar si la piedra puede moverse.

    Returns:
        bool: Devuelve si el jugador puede ir a esa coordenada.
    """
    return (is_rocks_free(board, rocks, coord) and is_cross_free(board, crosses, coord) and not tuple(coord) in water)


### > Funciones de movimiento

In [6]:
# Cargar Celda

def remove_crosses_from(crosses: StateObject,
                        enemies: StateObject,
                        coord: StaticObject):
    """
    Elimina las cruces de los enemigos dadas unas coordenadas.
    Los enemigos solo se colocan en vertical en ambos sentidos.

    Args:
        crosses (StateObject): Cruces.
        enemies (StateObject): Enemigos.
        coord (StaticObject): Coordenadas.

    Returns:
        StateObject: Nuevas cruces.
    """
    new_crosses = set(crosses)
    coord_x = coord[0]
    enemy = None
    for enem in enemies:
        if enem[1] == coord[1]:
            enemy = enem

    enemy_is_facing_down = enemy[0] < coord_x

    for cross in crosses:
        cross_x, cross_y = cross
        remove_crosses = ((enemy_is_facing_down and cross_x >= coord_x)
                    or
                    (not enemy_is_facing_down and cross_x <= coord_x))
        if cross_y == enemy[1] and remove_crosses:
            new_crosses.remove(cross)

    return new_crosses

In [7]:
# Cargar Celda

import numpy as np

def move_player(board: LevelBoard,
                player: StatePlayer,
                rocks: StateObject,
                water: StateObject,
                crosses: StatePlayer,
                key: StaticObject,
                target: StaticObject,
                mov: StatePlayer,
                player_has_key: bool) -> State:
    """
    Mueve el jugador.
    Arriba = (1, 0)
    Abajo = (-1, 0)
    Derecha = (0, 1)
    Izquierda = (0, -1)


    Args:
        board (LevelBoard): Tablero.
        player (StatePlayer): Jugador.
        rocks (StateObject): Rocas.
        water (StateObject): Agua.
        crosses (StatePlayer): Cruces.
        key (StaticObject): Llave.
        target (StaticObject): Objetivo.
        mov (StatePlayer): Movimiento.
        player_has_key (bool): Valor que indica si el jugador tiene la llave.

    Returns:
        State: Nuevo estado tras mover al jugador.
    """
    next_coord = tuple(np.add(player, mov))
    new_key = tuple(key)
    new_player = list(player)
    new_player_has_key = bool(player_has_key)

    if is_player_free(board, rocks, water, crosses, next_coord):
        if next_coord == target:
            # If players wins => Remove player.
            if player_has_key:
                new_player = None
        else:
            new_player = list(next_coord)
            # If players picks the key => Remove key.
            if next_coord == key:
                new_key = ()
                new_player_has_key = True

    return State(new_player, rocks, water, crosses, new_key, new_player_has_key)


def move_rocks(board: LevelBoard,
               player: StatePlayer,
               enemies: StatePlayer,
               rocks: StateObject,
               water: StateObject,
               crosses: StatePlayer,
               key: StaticObject,
               target: StaticObject,
               mov: StatePlayer,
               player_has_key: bool) -> State:
    """
    Mueve las rocas.


    Args:
        board (LevelBoard): Tablero.
        player (StatePlayer): Jugador.
        enemies (StateObject): Enemigos.
        rocks (StateObject): Rocas.
        water (StateObject): Agua.
        crosses (StatePlayer): Cruces.
        key (StaticObject): Llave.
        target (StaticObject): Objetivo.
        mov (StatePlayer): Movimiento.
        player_has_key (bool): Valor que indica si el jugador tiene la llave.

    Returns:
        State: Nuevo estado tras mover la roca.
    """
    next_coord = tuple(np.add(player, mov))
    next_rock_space = tuple(np.add(next_coord, mov))

    new_rocks = set(rocks)
    new_water = set(water)
    new_crosses = set(crosses)

    def rock_on_cross() -> bool:
        x, y = next_coord
        coord_in_crosses = (x - 1, y) in crosses or (x + 1, y) in crosses
        coord_in_enemy = (x - 1, y) in enemies or (x + 1, y) in enemies
        return (coord_in_crosses or coord_in_enemy) and tuple(mov) not in ((1, 0), (-1, 0))

    if (next_coord in rocks
            and is_rocks_free(board, rocks, next_rock_space)
        and next_rock_space != target
            and not rock_on_cross()):

        new_rocks.remove(next_coord)
        # We check if water in new rock position.
        if next_rock_space in water:
            new_water.remove(next_rock_space)
        else:
            if next_rock_space in crosses:
                new_crosses = remove_crosses_from(
                    crosses, enemies, next_rock_space)
            new_rocks.add(next_rock_space)

    return State(player, new_rocks, new_water, new_crosses, key, player_has_key)


def move(level: Level,
         state: State,
         mov: StatePlayer) -> State:
    """
    Mueve el jugador en el movimiento indicado y compruba si se está empujando alguna piedra.

    Args:
        level (Level): Nivel con los enemigos, el mapa y la meta.
        state (State): El estado de la partida.
        mov (StatePlayer): El movimiento a realizar.

    Returns:
        State: Retorna el estado tras mover al jugador y las piedras necesarias.
    """
    pos_movements = ((1, 0), (-1, 0), (0, 1), (0, -1))

    board = level.get_board()
    target = level.get_target()
    enemies = level.get_enemies()

    player = state.get_player()
    rocks = state.get_rocks()
    water = state.get_water()
    crosses = state.get_crosses()
    key = state.get_key()
    player_has_key = state.get_player_has_key()

    if tuple(mov) in pos_movements and player:
        s1 = move_rocks(
            board, player, enemies, rocks, water, crosses, key, target, mov, player_has_key)
        s2 = move_player(
            board, player, s1.get_rocks(), s1.get_water(), s1.get_crosses(), key, target, mov, player_has_key)
        return s2

    return state


### > Función de finalización

In [8]:
# Cargar Celda

def is_goal(state: State):
    """
    Devuelve si el estado es un estado final.

    Args:
        state (State): Estado a comprobar si es el último.

    Returns:
        bool: Devuelve partida terminada si el jugador no está.
    """
    player = state.get_player()
    return not player

## Salida
---

In [9]:
from IPython.display import display
from MediadorVPedro import Mediator
from Gui import Gui

import warnings
warnings.filterwarnings("ignore")
ui = Gui(manual = True)

med = Mediator.get_instance(modelPath = "JuegoEscapaManual.ipynb")   
med.register_ui(ui)


display(ui.get_ui_elements())

<module 'Model'>
